In [1]:
# Cell 1: Imports and Setup
# This cell contains the necessary import statements for the required libraries and modules.
# It also includes any initial setup or configuration needed for the program to run correctly.
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")
print("Ready to build your Stock Screener!")

✅ All libraries imported successfully!
Ready to build your Stock Screener!


In [2]:
# ============================================================================
# CELL 2: Define Stock Tickers
# ============================================================================

# List of popular stock tickers 
STOCK_TICKERS = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'BRK-B', 'UNH', 'JNJ',
    'V', 'WMT', 'JPM', 'MA', 'PG', 'CVX', 'HD', 'PFE', 'BAC', 'ABBV',
    'KO', 'AVGO', 'PEP', 'TMO', 'COST', 'MRK', 'ACN', 'DHR', 'VZ', 'ABT',
    'ADBE', 'LLY', 'CRM', 'TXN', 'NKE', 'NFLX', 'AMD', 'QCOM', 'NEE', 'BMY',
    'UPS', 'PM', 'T', 'LOW', 'ORCL', 'MDT', 'UNP', 'IBM', 'CAT', 'GS'
]

print(f"📊 We'll analyze {len(STOCK_TICKERS)} stocks:")
print(STOCK_TICKERS[:10], "... and more")

📊 We'll analyze 50 stocks:
['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'BRK-B', 'UNH', 'JNJ'] ... and more


In [3]:
# ============================================================================
# CELL 3: Define Data Collection Functions
# ============================================================================

def get_stock_data(ticker):
    """Get comprehensive stock data for analysis"""
    try:
        stock = yf.Ticker(ticker)
        
        # Get basic info
        info = stock.info
        
        # Get historical data for calculations
        hist = stock.history(period="1y")
        
        if len(hist) == 0:
            return None
            
        # Calculate metrics
        current_price = hist['Close'].iloc[-1]
        year_high = hist['High'].max()
        year_low = hist['Low'].min()
        
        # Calculate returns and volatility
        returns = hist['Close'].pct_change().dropna()
        annual_return = ((current_price / hist['Close'].iloc[0]) - 1) * 100
        volatility = returns.std() * np.sqrt(252) * 100  # Annualized volatility
        
        # Extract key metrics
        data = {
            'ticker': ticker,
            'company_name': info.get('longName', ticker),
            'current_price': current_price,
            'market_cap': info.get('marketCap', 0),
            'pe_ratio': info.get('trailingPE', None),
            'forward_pe': info.get('forwardPE', None),
            'peg_ratio': info.get('pegRatio', None),
            'debt_to_equity': info.get('debtToEquity', None),
            'roe': info.get('returnOnEquity', None),
            'profit_margin': info.get('profitMargins', None),
            'revenue_growth': info.get('revenueGrowth', None),
            'annual_return': annual_return,
            'volatility': volatility,
            'year_high': year_high,
            'year_low': year_low,
            'price_to_high_ratio': (current_price / year_high) * 100,
            'dividend_yield': info.get('dividendYield', 0) * 100 if info.get('dividendYield') else 0,
            'beta': info.get('beta', None),
            'sector': info.get('sector', 'Unknown')
        }
        
        return data
        
    except Exception as e:
        print(f"❌ Error fetching data for {ticker}: {e}")
        return None

print("✅ Data collection functions defined!")

✅ Data collection functions defined!


In [4]:
# ============================================================================
# CELL 4: Collect Stock Data (This will take 3-5 minutes)
# ============================================================================

print("🔄 Collecting stock data... This may take a few minutes.")
print("Grab a coffee ☕ while we fetch real-time financial data!")

stock_data = []
for i, ticker in enumerate(STOCK_TICKERS):
    print(f"Processing {ticker} ({i+1}/{len(STOCK_TICKERS)})", end=" ")
    data = get_stock_data(ticker)
    if data:
        stock_data.append(data)
        print("✅")
    else:
        print("❌")

print(f"\n🎉 Successfully collected data for {len(stock_data)} stocks!")


🔄 Collecting stock data... This may take a few minutes.
Grab a coffee ☕ while we fetch real-time financial data!
Processing AAPL (1/50) ✅
Processing MSFT (2/50) ✅
Processing GOOGL (3/50) ✅
Processing AMZN (4/50) ✅
Processing TSLA (5/50) ✅
Processing META (6/50) ✅
Processing NVDA (7/50) ✅
Processing BRK-B (8/50) ✅
Processing UNH (9/50) ✅
Processing JNJ (10/50) ✅
Processing V (11/50) ✅
Processing WMT (12/50) ✅
Processing JPM (13/50) ✅
Processing MA (14/50) ✅
Processing PG (15/50) ✅
Processing CVX (16/50) ✅
Processing HD (17/50) ✅
Processing PFE (18/50) ✅
Processing BAC (19/50) ✅
Processing ABBV (20/50) ✅
Processing KO (21/50) ✅
Processing AVGO (22/50) ✅
Processing PEP (23/50) ✅
Processing TMO (24/50) ✅
Processing COST (25/50) ✅
Processing MRK (26/50) ✅
Processing ACN (27/50) ✅
Processing DHR (28/50) ✅
Processing VZ (29/50) ✅
Processing ABT (30/50) ✅
Processing ADBE (31/50) ✅
Processing LLY (32/50) ✅
Processing CRM (33/50) ✅
Processing TXN (34/50) ✅
Processing NKE (35/50) ✅
Processing NFL

In [5]:
# ============================================================================
# CELL 5: Create and Clean DataFrame
# ============================================================================

# Create DataFrame
df = pd.DataFrame(stock_data)

print("📊 Raw data shape:", df.shape)

# Clean the data
df_clean = df.dropna(subset=['pe_ratio', 'market_cap'])
df_clean = df_clean[df_clean['market_cap'] > 0]
df_clean = df_clean[df_clean['pe_ratio'] > 0]
df_clean = df_clean[df_clean['pe_ratio'] < 100]  # Remove extreme PE ratios

print("🧹 Cleaned data shape:", df_clean.shape)
print(f"✨ Final dataset has {len(df_clean)} stocks ready for analysis!")


📊 Raw data shape: (50, 19)
🧹 Cleaned data shape: (47, 19)
✨ Final dataset has 47 stocks ready for analysis!


In [6]:
# ============================================================================
# CELL 6: Explore the Data
# ============================================================================

# Display basic info
print("📈 STOCK SCREENER DATA PREVIEW")
print("=" * 50)

# Show sample data
display_cols = ['ticker', 'company_name', 'current_price', 'pe_ratio', 'market_cap', 'annual_return']
print(df_clean[display_cols].head(10))

print("\n📊 DATASET STATISTICS")
print("=" * 30)
print(f"Total Stocks: {len(df_clean)}")
print(f"Average PE Ratio: {df_clean['pe_ratio'].mean():.2f}")
print(f"Average Annual Return: {df_clean['annual_return'].mean():.2f}%")
print(f"Sectors Covered: {df_clean['sector'].nunique()}")


📈 STOCK SCREENER DATA PREVIEW
   ticker                     company_name  current_price   pe_ratio  \
0    AAPL                       Apple Inc.     232.139999  35.226100   
1    MSFT            Microsoft Corporation     506.690002  37.174614   
2   GOOGL                    Alphabet Inc.     212.910004  22.674122   
3    AMZN                 Amazon.com, Inc.     229.000000  34.961830   
5    META             Meta Platforms, Inc.     738.700012  26.813068   
6    NVDA               NVIDIA Corporation     174.179993  49.463352   
7   BRK-B          Berkshire Hathaway Inc.     502.980011  17.243060   
8     UNH  UnitedHealth Group Incorporated     309.869995  13.420095   
9     JNJ                Johnson & Johnson     177.169998  18.968950   
10      V                        Visa Inc.     351.779999  34.353516   

       market_cap  annual_return  
0   3445050310656       1.842964  
1   3766312763392      22.379535  
2   2578297651200      30.947089  
3   2442261954560      28.291317  
5 

In [7]:
# ============================================================================
# CELL 7: Save Data
# ============================================================================

# Save to CSV for later use
df_clean.to_csv('stock_data.csv', index=False)
print("💾 Data saved to 'stock_data.csv'")
print("🚀 Ready for Phase 2 - Building the ML Model!")

# Make df_clean available for next phases
stock_df = df_clean.copy()
print("✅ Data stored in 'stock_df' variable for ML modeling")

💾 Data saved to 'stock_data.csv'
🚀 Ready for Phase 2 - Building the ML Model!
✅ Data stored in 'stock_df' variable for ML modeling


In [8]:
# ============================================================================
# CELL 8: Import ML Libraries
# ============================================================================

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Set up plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ ML libraries imported successfully!")


Matplotlib is building the font cache; this may take a moment.


✅ ML libraries imported successfully!


In [12]:
# ============================================================================
# CELL 9: Feature Engineering & Target Creation (FIXED)
# ============================================================================

def create_features_and_target(df):
    """Create ML features and target variable"""
    
    # Create a copy to avoid modifying original
    ml_df = df.copy()
    
    print(f"🔧 Starting with {len(ml_df)} stocks")
    
    # Fill missing values with median for numeric columns
    numeric_cols = ['pe_ratio', 'debt_to_equity', 'roe', 'profit_margin', 
                   'revenue_growth', 'beta', 'peg_ratio', 'forward_pe']
    
    for col in numeric_cols:
        if col in ml_df.columns:
            ml_df[col] = ml_df[col].fillna(ml_df[col].median())
    
    # Ensure key columns exist and are not all NaN
    required_cols = ['annual_return', 'volatility', 'pe_ratio']
    for col in required_cols:
        if col not in ml_df.columns or ml_df[col].isna().all():
            print(f"❌ Missing required column: {col}")
            return None, None, None, None
    
    # Create composite target score (what we want to predict/rank)
    # Higher score = better investment opportunity
    
    # Normalize returns (higher is better) - handle NaN and infinite values
    returns_clean = ml_df['annual_return'].replace([np.inf, -np.inf], np.nan).fillna(0)
    if returns_clean.max() != returns_clean.min():
        returns_score = (returns_clean - returns_clean.min()) / (returns_clean.max() - returns_clean.min())
    else:
        returns_score = pd.Series([0.5] * len(ml_df))
    
    # Normalize volatility (lower is better, so invert)
    volatility_clean = ml_df['volatility'].replace([np.inf, -np.inf], np.nan).fillna(ml_df['volatility'].median())
    if volatility_clean.max() != volatility_clean.min():
        volatility_score = 1 - ((volatility_clean - volatility_clean.min()) / (volatility_clean.max() - volatility_clean.min()))
    else:
        volatility_score = pd.Series([0.5] * len(ml_df))
    
    # PE ratio score (lower is generally better)
    pe_clean = ml_df['pe_ratio'].replace([np.inf, -np.inf], np.nan).fillna(ml_df['pe_ratio'].median())
    if pe_clean.max() != pe_clean.min():
        pe_score = 1 - ((pe_clean - pe_clean.min()) / (pe_clean.max() - pe_clean.min()))
    else:
        pe_score = pd.Series([0.5] * len(ml_df))
    
    # ROE score (higher is better)
    if 'roe' in ml_df.columns and ml_df['roe'].notna().sum() > 0:
        roe_clean = ml_df['roe'].replace([np.inf, -np.inf], np.nan).fillna(0)
        if roe_clean.max() != roe_clean.min() and roe_clean.max() > 0:
            roe_score = (roe_clean - roe_clean.min()) / (roe_clean.max() - roe_clean.min())
        else:
            roe_score = pd.Series([0.5] * len(ml_df))
    else:
        roe_score = pd.Series([0.5] * len(ml_df))  # neutral if no data
    
    # Combine scores (you can adjust weights)
    ml_df['investment_score'] = (
        0.3 * returns_score + 
        0.25 * volatility_score + 
        0.25 * pe_score + 
        0.2 * roe_score
    )
    
    # Remove any remaining NaN or infinite values in target
    ml_df['investment_score'] = ml_df['investment_score'].replace([np.inf, -np.inf], np.nan)
    ml_df = ml_df.dropna(subset=['investment_score'])
    
    print(f"🧹 After cleaning: {len(ml_df)} stocks with valid scores")
    
    # Select features for ML model
    feature_cols = ['pe_ratio', 'market_cap', 'annual_return', 'volatility', 
                   'price_to_high_ratio', 'dividend_yield']
    
    # Add optional features if available and have enough data
    optional_features = ['debt_to_equity', 'roe', 'profit_margin', 'beta']
    for feat in optional_features:
        if feat in ml_df.columns and ml_df[feat].notna().sum() > len(ml_df) * 0.3:
            feature_cols.append(feat)
    
    # Final cleaning of features
    X = ml_df[feature_cols].copy()
    for col in feature_cols:
        X[col] = X[col].replace([np.inf, -np.inf], np.nan).fillna(X[col].median())
    
    y = ml_df['investment_score']
    
    # Final check for any remaining issues
    if X.isna().any().any():
        print("⚠️ Still have NaN in features, filling with 0")
        X = X.fillna(0)
    
    if y.isna().any():
        print("⚠️ Still have NaN in target, removing these rows")
        valid_idx = ~y.isna()
        X = X[valid_idx]
        y = y[valid_idx]
        ml_df = ml_df[valid_idx]
    
    print(f"📊 Final dataset: {len(X)} stocks with {len(feature_cols)} features")
    print("Features:", feature_cols)
    print(f"🎯 Target variable: investment_score (range: {y.min():.3f} - {y.max():.3f})")
    
    return X, y, ml_df, feature_cols

# Run feature engineering
X, y, ml_df, feature_cols = create_features_and_target(stock_df)
print(f"\n✅ Feature engineering complete!")
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

🔧 Starting with 47 stocks
🧹 After cleaning: 47 stocks with valid scores
📊 Final dataset: 47 stocks with 10 features
Features: ['pe_ratio', 'market_cap', 'annual_return', 'volatility', 'price_to_high_ratio', 'dividend_yield', 'debt_to_equity', 'roe', 'profit_margin', 'beta']
🎯 Target variable: investment_score (range: 0.141 - 0.741)

✅ Feature engineering complete!
Features shape: (47, 10)
Target shape: (47,)


In [10]:
# ============================================================================
# CELL 10: Train ML Model
# ============================================================================

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest model
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

print("🤖 Training Random Forest model...")
rf_model.fit(X_train_scaled, y_train)

# Evaluate model
train_score = rf_model.score(X_train_scaled, y_train)
test_score = rf_model.score(X_test_scaled, y_test)

print(f"✅ Model trained successfully!")
print(f"📈 Training R² Score: {train_score:.3f}")
print(f"🎯 Test R² Score: {test_score:.3f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n🔍 Most Important Features:")
for idx, row in feature_importance.head().iterrows():
    print(f"  {row['feature']}: {row['importance']:.3f}")

🤖 Training Random Forest model...


ValueError: Input y contains NaN.